In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

In [3]:
# check CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1) # (28, 28)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)   # (28, 28)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # (28, 28)->(14, 14)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)  # (14, 14)
        self.bn4 = nn.BatchNorm2d(512)

        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)  # （14， 14） -> (7, 7)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6 = nn.Conv2d(512, 256, kernel_size=3, padding=1)  # (7, 7)
        self.bn6 = nn.BatchNorm2d(256)

        self.conv7 = nn.Conv2d(256, 128, kernel_size=3, padding=1)  #  (7, 7)
        self.bn7 = nn.BatchNorm2d(128)

        self.conv8 = nn.Conv2d(128, 32, kernel_size=3, padding=1)  #  (7, 7)
        self.bn8 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout(0.2)

        self.fc1 = nn.Linear(32 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 15)  # 15 output classes

    def forward(self, x):
        x = (F.relu(self.bn1(self.conv1(x))))   #1
        x = (F.relu(self.bn2(self.conv2(x))))   #2
        #x = self.dropout(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  #3
        x = (F.relu(self.bn4(self.conv4(x))))   #4
        x = self.pool(F.relu(self.bn5(self.conv5(x))))  #5
        x = self.dropout(x)
        x = (F.relu(self.bn6(self.conv6(x))))   #6
        x = (F.relu(self.bn7(self.conv7(x))))   #7
        x = (F.relu(self.bn8(self.conv8(x))))   #8

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
import zipfile
import os

# First, unzip the file
#with zipfile.ZipFile('../data.npz.zip', 'r') as zip_ref:
#    zip_ref.extractall()  # You can specify a directory path here if needed


In [6]:

# load the dataset 

data = np.load('data.npz')
X_train = data['X_train'].astype(np.float32)  # Shape: (82875, 784)
y_train = data['y_train'].astype(np.int64)    # Shape: (82875,)
X_test = data['X_test'].astype(np.float32)    # Shape: (14625, 784)


In [7]:
# Normalize the data
X_scaled = X_train/255
X_scaled = X_scaled.reshape(-1, 1, 28, 28)  # Reshape back to (n_samples, channels, height, width)

X_test_scaled = X_test/255
X_test_scaled = X_test_scaled.reshape(-1, 1, 28, 28)

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_train, test_size=0.2, random_state=42)

In [9]:
# Convert arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val)
X_test_tensor = torch.tensor(X_test_scaled)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
# Initialize the model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
#optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9, eps=1e-09,weight_decay=0,momentum=0.9,centered=False)

In [11]:
# Training model
def train(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device) # Move data to GPU if available
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


# Validate the model
def validate_model():
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)  # Move data to GPU if available
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

# Prediction function
def predict(data_loader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            outputs = model(data)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
    return all_preds

In [69]:
# Run the training and validation
train(20)
validate_model()

Epoch 1, Loss: 0.14749841392040253
Epoch 2, Loss: 0.03512042015790939
Epoch 3, Loss: 0.12077698856592178
Epoch 4, Loss: 0.0295538492500782
Epoch 5, Loss: 0.04331273213028908
Epoch 6, Loss: 0.040117621421813965
Epoch 7, Loss: 0.03122521936893463
Epoch 8, Loss: 0.08467192202806473
Epoch 9, Loss: 0.0029725513886660337
Epoch 10, Loss: 0.12311655282974243
Epoch 11, Loss: 0.004145391751080751
Epoch 12, Loss: 0.009853522293269634
Epoch 13, Loss: 0.050638582557439804
Epoch 14, Loss: 0.011532285250723362
Epoch 15, Loss: 0.011622577905654907
Epoch 16, Loss: 0.002295181853696704
Epoch 17, Loss: 0.012755406089127064
Epoch 18, Loss: 0.00031205645063892007
Epoch 19, Loss: 0.0003736028738785535
Epoch 20, Loss: 0.14515523612499237
Validation Accuracy: 96.94%


In [70]:
# Predict on the test set
test_loader = DataLoader(X_test_tensor, batch_size=64, shuffle=False)
test_predictions = predict(test_loader)

print(np.shape(test_predictions))

(14625,)


In [71]:
# # Save predictions
# predict_id = np.arange(0, len(X_test))
# submission_predictions = np.vstack((predict_id, test_predictions)).T

# # Save the predictions to a CSV file
# np.savetxt("cnn_predictions.csv",submission_predictions, delimiter=",", fmt='%d', header="ID,Label", comments='')


In [72]:
from datetime import datetime
import os

# Assume test_predictions are available from your model's output

# Create the submission directory if it doesn't exist
directory = "submission"
if not os.path.exists(directory):
    os.makedirs(directory)

# Get current date and time
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"cnn_predictions_{current_time}.csv"

# Full path for saving the file
full_path = os.path.join(directory, filename)

# Preparing the data to save
predict_id = np.arange(0, len(X_test))
submission_predictions = np.vstack((predict_id, test_predictions)).T

# Save the predictions to a CSV file
np.savetxt(full_path, submission_predictions, delimiter=",", fmt='%d', header="ID,Label", comments='')

print(f"File saved as {full_path}")

File saved as submission/cnn_predictions_20240414_231356.csv


In [73]:
# from google.colab import files

# files.download('cnn_predictions.csv')

In [15]:
from torchsummary import summary
input_shape = (1, 28, 28)
summary(model, input_shape)
# print(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
       BatchNorm2d-2           [-1, 64, 28, 28]             128
            Conv2d-3          [-1, 128, 28, 28]          73,856
       BatchNorm2d-4          [-1, 128, 28, 28]             256
            Conv2d-5          [-1, 256, 28, 28]         295,168
       BatchNorm2d-6          [-1, 256, 28, 28]             512
         MaxPool2d-7          [-1, 256, 14, 14]               0
            Conv2d-8          [-1, 512, 14, 14]       1,180,160
       BatchNorm2d-9          [-1, 512, 14, 14]           1,024
           Conv2d-10          [-1, 512, 14, 14]       2,359,808
      BatchNorm2d-11          [-1, 512, 14, 14]           1,024
        MaxPool2d-12            [-1, 512, 7, 7]               0
          Dropout-13            [-1, 512, 7, 7]               0
           Conv2d-14            [-1, 25